# Lectures 13-15 

- extracts from lectures 13-15
- 16 will be covered in next course
- good to have some extra tools in your toolkit
- connecting duckdb to python 

In [2]:
with open("sql/ingestion.sql", 'r') as sql_script:
    sql_code = sql_script.read()

sql_code

"CREATE SCHEMA IF NOT EXISTS staging;\n\nCREATE TABLE IF NOT EXISTS staging.students AS (\n\tSELECT * FROM read_csv_auto('data/students.csv')\n);\n\nCREATE TABLE IF NOT EXISTS staging.teachers AS (\n\tSELECT * FROM read_csv_auto('data/teachers.csv')\n);\n\nCREATE TABLE IF NOT EXISTS staging.housing_sales AS (\n\tSELECT * FROM read_csv_auto('data/housing_sales.csv')\n);\n\nCREATE TABLE IF NOT EXISTS staging.neighborhoods AS (\n\tSELECT * FROM read_csv_auto('data/neighborhoods.csv')\n);\n\nCREATE TABLE IF NOT EXISTS staging.categories AS (\n\tSELECT * FROM read_csv_auto('data/categories.csv')\n);\n\nCREATE TABLE IF NOT EXISTS staging.sales AS (\n\tSELECT * FROM read_csv_auto('data/sales.csv')\n);"

## with statement to connect to a duckdb

- with handles connection and closes it down properly
- if this database doesn't exist -> it creates it for us

In [3]:
import duckdb

with duckdb.connect("data/lecture_13_15.duckdb") as conn:
    conn.execute(sql_code)
    describe = conn.execute("DESC;").df()

describe

,database,schema,name,column_names,column_types,temporary
0,lecture_13_15,staging,categories,"[category_id, category_name, parent_category_id]","[BIGINT, VARCHAR, BIGINT]",False
1,lecture_13_15,staging,housing_sales,"[sale_id, neighborhood_id, address, date_of_sa...","[VARCHAR, BIGINT, VARCHAR, DATE, VARCHAR, BIGI...",False
2,lecture_13_15,staging,neighborhoods,"[neighborhood_id, name, city, postal_code, med...","[BIGINT, VARCHAR, VARCHAR, BIGINT, BIGINT, BIG...",False
3,lecture_13_15,staging,sales,"[product_id, product_name, category_id, quanti...","[VARCHAR, VARCHAR, BIGINT, BIGINT, BIGINT]",False
4,lecture_13_15,staging,students,"[student_id, grade_math, class_name]","[VARCHAR, BIGINT, VARCHAR]",False
5,lecture_13_15,staging,teachers,"[class_name, teacher_name]","[VARCHAR, VARCHAR]",False


In [5]:
# this doesn't work as the connection is closed
# duckdb.sql("FROM staging.students;")

In [6]:
import pandas as pd


def query_database(
    query: str, db_path: str = "data/lecture_13_15.duckdb"
) -> pd.DataFrame:
    """convenience function to open a duckdb database connection and query it"""
    with duckdb.connect(db_path) as conn:
        return conn.execute(query=query).df()

query_database("FROM staging.students;")

,student_id,grade_math,class_name
0,S001,92,A
1,S002,85,A
2,S003,78,B
3,S004,88,B
4,S005,95,C
5,S006,67,C
6,S007,81,D
7,S008,74,D
8,S009,89,E
9,S010,93,E
